### 機能要件
- 2値の分類（Classification）タスクを扱える
- カテゴリカル変数を指定するとone-hotエンコードを実行する
- モデル用データマートに施したのと同一データ前処理をスコア用データマートに対しても適用される
- モデル選択の評価指標を選択できる
- 複数アルゴリズムから指定の評価指標に従いベストモデルを選択できる
- 学習済みモデルを保存できる
- アルゴリズムランキングと性能評価指標が出力される
- 学習済みモデル（保存したモデル）を呼び出しスコア用データに対し予測確率を付与できる

## データの読み込み

- ### 正解データが組み込まれたcsvファイル

In [24]:
# csvには読み込むcsvファイルのパスを指定して下さい
def read_data_contain_test(csv='./data/av_loan_u6lujuX_CVtuZ9i.csv'):
    import pandas as pd
    # csvファイルから読み込み
    df = pd.read_csv(csv, header=0)
    # 一番最後の列を正解データとする
    X = df.iloc[:,:-1]
    print('特徴量データXの確認:')
    display(X.head())
    y = df.iloc[:,-1]
    print('正解データyの確認:')
    display(y.head())
    # shapeの確認
    print(f'X shape: {X.shape}')
    print(f'y shape: {y.shape}')
    return X, y
    
X, y = read_data_contain_test()

特徴量データXの確認:


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban


正解データyの確認:


0    Y
1    N
2    Y
3    Y
4    Y
Name: Loan_Status, dtype: object

X shape: (614, 12)
y shape: (614,)


- ###  訓練データとテストデータで分割して渡される場合

In [27]:
# csvには読み込むcsvファイルのパスを指定して下さい
def read_data_not_contain_test(csv='./data/av_loan_u6lujuX_CVtuZ9i.csv', csv_test='./data/av_loan_test_Y3wMUE5_7gLdaTN.csv'):
    import pandas as pd
    
    # csvファイルから読み込み
    df = pd.read_csv(csv, header=0)
    df_test = pd.read_csv(csv_test, header=0)
    
    # 一番最後の列以外をXとする
    X = df.iloc[:,:-1]
    X_test = df_test.iloc[:,:-1]
    
    # 確認
    print('特徴量データXの確認:')
    display(X.head())
    print('特徴量データX_testの確認:')
    display(X_test.head())
    
    # 一番最後の列をyとする
    y = df.iloc[:,-1]
    y_test = df_test.iloc[:,-1]
    
    # 確認
    print('正解データyの確認:')
    display(y.head())
    print('正解データy_testの確認:')
    display(y_test.head())
    
    # shapeの確認
    print(f'X shape: {X.shape}')
    print(f'y shape: {y.shape}')
    print(f'X_test shape: {X_test.shape}')
    print(f'y_test shape: {y_test.shape}')
    return X, X_test, y, y_test
    
X, X_test, y, y_test = read_data_not_contain_test()

特徴量データXの確認:


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban


特徴量データX_testの確認:


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
0,LP001015,Male,Yes,0.0,Graduate,No,5720,0,110.0,360.0,1.0
1,LP001022,Male,Yes,1.0,Graduate,No,3076,1500,126.0,360.0,1.0
2,LP001031,Male,Yes,2.0,Graduate,No,5000,1800,208.0,360.0,1.0
3,LP001035,Male,Yes,2.0,Graduate,No,2340,2546,100.0,360.0,NaN
4,LP001051,Male,No,0.0,Not Graduate,No,3276,0,78.0,360.0,1.0


正解データyの確認:


0    Y
1    N
2    Y
3    Y
4    Y
Name: Loan_Status, dtype: object

正解データy_testの確認:


0    Urban
1    Urban
2    Urban
3    Urban
4    Urban
Name: Property_Area, dtype: object

X shape: (614, 12)
y shape: (614,)
X_test shape: (333, 11)
y_test shape: (333,)


## 前処理

### ホールド・アウト法によるデータの分割

In [26]:
# Xには特徴データ,yには正解データ,test_sizeは分割割合をを代入してください
X = X
y = y

def hold_out(X, y, test_size=0.20):
    from sklearn.model_selection import train_test_split
    # ホールドアウト法
    # 訓練データを80%テストデータを20%に分割する
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)
    print('X_train, X_test', X_train.shape, X_test.shape)
    print('y_train, y_test', y_train.shape, y_test.shape)
    
hold_out()

NameError: name 'X' is not defined